In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf
import os
import random
import datetime
#from surprise import BaselineOnly
#from surprise import Dataset
#from surprise import evaluate
from surprise import Reader
import collections as coll

In [20]:
def generate_rec(predictions):
    rec = coll.defaultdict(list)

    for p in predictions:
        uid, iid, est = p.uid, p.iid, p.est
        #print uid, iid, est

        rec[uid].append((iid, est))

    for u in rec:
        rec[u].sort(key=lambda x: -1 * x[1])

    for u in rec:
        #print rec[u]
        #break

        rec[u] = rec[u][:N]
    return rec

In [2]:
def calculate_metrics(test, rec):
    starttime = datetime.datetime.now()
    hit = 0

    all__for_recall = 0
    all__for_precision = 0
    for user in test.keys():
        history = test[user][0]
        answer = test[user][1]
        tu = [x[0] for x in answer]
        rank = rec[user] # self.recommend(history, N)
        #print 'rank:', rank
        for item, pui in rank:
            if item in tu:
                hit += 1
        all__for_recall += len(tu)
        all__for_precision += len(rank) #Note: In book RSP, the author used 'all += N'

    metric_recall = None
    metric_precision = None
    metric_f1 = None
    if 0 == all__for_recall:
        metric_recall = 0
    else:
        metric_recall = hit / (all__for_recall * 1.0)

    if 0 == all__for_precision:
        metric_precision = 0
    else:
        metric_precision = hit / (all__for_precision * 1.0)

    if 0 == all__for_recall or 0 == all__for_precision:
        metric_f1 = 0
    else:
        metric_f1 = 2/(1./metric_precision + 1./metric_recall)

    endtime = datetime.datetime.now()
    interval = (endtime - starttime).seconds
    print 'metric calculation: time consumption: %d' % (interval)
    return {'recall': metric_recall, 'precision': metric_precision, 'f1': metric_f1}

In [3]:

def extract_data_from_file_and_generate_train_and_test(filename, train_percent, seed, delimiter, test_data_inner_ratio, sort_by_time=False):
    test = None
    train = None
    data = {}
    random.seed(seed)

    with open(filename , 'r') as f:
        first_line = f.readline()
        for i, line in enumerate(f):
            userId, movieId, rating, timestamp = line.split(delimiter)
            #userId = int(userId)
            #movieId = int(movieId)
            rating = float(rating)
            timestamp = int(timestamp)

            if userId not in data:
                data[userId] = []
            data[userId].append((movieId, rating, timestamp))

    test = {}
    train = {}
    for userId in data:
        total_len = len(data[userId])
        if random.random() >= train_percent:
        #if 0 == random.randint(0, 2):
        #if 2 == random.randint(0, 2):
            test[userId] = data[userId]
        else:
            train[userId] = data[userId]
    userId = None

    for userId in test:
        test[userId].sort(key=lambda x: x[2])
    # sort by time: PART 1<begin>
    if sort_by_time:
        print 'sort by time PART 1'
        for userId in train:
            train[userId].sort(key=lambda x: x[2])

    #print train[train.keys()[0]]
    #print test[test.keys()[0]]
    #raw_input()
    # sort by time: PART 1 <end>

    ### split test data further
    test_real = {}
#    if not sort_by_time:
#        for k_user in test:
#            test_real[k_user] = [[], []]
#            for m, r, t in test[k_user]:
#                # every record of test dataset is supposed to be splitted into 2 parts: input part and fact/answer part
#                # How to specify the relative of these parts? 
#                #  Assign num_of_parts_of_test_data an appropriate value: each record of test dataset is supposed
#                # to constitute num_of_parts_of_test_data parts, and one of them would serve as the fact/answer part.
#                if 0 == random.randint(0, num_of_parts_of_test_data):
#                    test_real[k_user][1].append((m, r, t)) # the fact/answer part in one record of test dataset
#                else:
#                    test_real[k_user][0].append((m, r, t)) # the input part in one record of test dataset
#    else: 
#        # sort by time: PART 2 <start>
#        print 'sort by time PART 2'
    for k_user in test:
        #print 'len(test[k_user]):', len(test[k_user])
        #print 'num_of_parts_of_test_data:', num_of_parts_of_test_data
        #raw_input()
        #print (len(test[k_user]) * (1.0 / num_of_parts_of_test_data))
        #print (int)(len(test[k_user]) * (1.0 / num_of_parts_of_test_data))
        split_point_index = -1 * ((int)(len(test[k_user]) * test_data_inner_ratio))
        #print 'split_point_index:', split_point_index
        test_real[k_user] = [test[k_user][:split_point_index], test[k_user][split_point_index:]]
        #raw_input()

        # sort by time: PART 2 <end>

    #print test_real[test_real.keys()[0]]
    print 'sort_by_time:', sort_by_time

    #raw_input('pause')

    return train, test_real


In [19]:
data_filename, delimiter, data_set = os.path.sep.join(['ml-1m', 'ratings.dat']), '::', '1M'
#data_filename, delimiter = os.path.sep.join(['ml-10M100K', 'ratings.dat']), '::'
#data_filename, delimiter, data_set = os.path.sep.join(['ml-100k', 'u.data']), '\t', '100K'

seed = 2 
K = 10
N = 20
train_percent = 0.8
test_data_inner_ratio = 0.8
train = None
test = None
original_train, original_test = extract_data_from_file_and_generate_train_and_test(data_filename, train_percent, seed, delimiter, test_data_inner_ratio)
#original_train, original_test = extract_data_from_file_and_generate_train_and_test(data_filename, 3, 0, seed, delimiter)

sort_by_time: False


In [9]:
train_file_name = 'tmp__svd_train'
test_file_name = 'tmp__svd_test'
my_sep = ':'

with open(train_file_name , 'w') as f:
     # format: userId, movieId, rating, timestamp
    for userId in original_train:
        for movieId, rating, timestamp in original_train[userId]:
            f.write(my_sep.join(map(str, [userId, movieId, rating, timestamp])) + '\n')
    for userId in original_test:
        for movieId, rating, timestamp in original_test[userId][0]:
            f.write(my_sep.join(map(str, [userId, movieId, rating, timestamp])) + '\n')

with open(test_file_name, 'w') as f:
    for userId in original_test:
        for movieId, rating, timestamp in original_test[userId][1]:
            f.write(my_sep.join(map(str, [userId, movieId, rating, timestamp])) + '\n')

In [13]:
reader = Reader(line_format='user item rating timestamp', sep=my_sep)

data = Dataset.load_from_folds([(train_file_name, test_file_name)], reader=reader)

for trainset, testset in data.folds():
    first_train, first_test = trainset, testset
    
print 'info of train:'
print first_train.n_users
print first_train.n_items
print first_train.n_ratings

print 'info of test:'
print len(first_test)

info of train:
6040
3698
840387
info of test:
159821


In [14]:
all_user_in_test = set([x[0] for x in first_test])
print len(all_user_in_test)


def extract_all_items(filename, delimiter):
    all_items = []
    with open(filename , 'r') as f:
        first_line = f.readline()
        for i, line in enumerate(f):
            userId, movieId, rating, timestamp = line.split(delimiter)
            
            all_items.append(movieId)
    return set(all_items)

all_item_in_test = None
all_items = extract_all_items(data_filename, delimiter)
print len(all_items)

1200
3706


In [17]:
#user_item_interaction = {}
#def extract_all_user_item_interaction(filename, delimiter):
#    ret = coll.defaultdict(set)
#    with open(filename , 'r') as f:
#        first_line = f.readline()
#        for i, line in enumerate(f):
#            userId, movieId, rating, timestamp = line.split(delimiter)
#            
#            ret[userId].add(movieId)
#    return ret
#
#user_item_interaction = extract_all_user_item_interaction(data_filename, delimiter)

user_item_interaction = None
user_item_interaction_in_history = {}#coll.defaultdict(dict)
for u in original_train:
    user_item_interaction_in_history[u] = set([x[0] for x in original_train[u]])
for u in original_test:
    user_item_interaction_in_history[u] = set([x[0] for x in original_test[u][0]])
    

In [18]:
# train SVD

starttime = datetime.datetime.now()

###
# We'll use the famous SVD algorithm.
algo = SVD()

# train algorithm.
algo.train(first_train)

###
endtime = datetime.datetime.now()
interval = (endtime - starttime).seconds
print 'svd time consumption: %d' % (interval)


svd time consumption: 38


In [28]:
uid = None

In [29]:
#predictions = algo.test([('4', '13', -1)])
predictions_without_filtering = None

mid = [(uid, iid, -1) for iid in all_items for uid in all_user_in_test]
predictions_without_filtering = algo.test(mid)
mid = None


In [ ]:
rec_without_filtering = generate_rec(predictions_without_filtering)

In [34]:
calculate_metrics(original_test, rec_without_filtering)

metric calculation: time consumption: 0


{'f1': 0.019725711425789216,
 'precision': 0.07554166666666666,
 'recall': 0.011343941034031822}

In [38]:
uid = None
#predictions = algo.test([('4', '13', -1)])
predictions_with_filtering = None

#mid = [(uid, iid, -1) for iid in filter(lambda x: x not in user_item_interaction_in_history[uid], all_items) for uid in all_user_in_test]
mid = []
for uid in all_user_in_test:
    mid += [(uid, iid, -1) for iid in filter(lambda x: x not in user_item_interaction_in_history[uid], all_items)]

In [40]:
print len(mid)

4406656


In [41]:
predictions_with_filtering = algo.test(mid)
mid = None

In [43]:
rec_with_filtering = generate_rec(predictions_with_filtering)
calculate_metrics(original_test, rec_with_filtering)

metric calculation: time consumption: 0


{'f1': 0.022369587805528203,
 'precision': 0.08566666666666667,
 'recall': 0.012864392038593176}